In [1]:
# Celda para ver cuanta ram tienes

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

print(ram_gb)

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!
89.639657472


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!pip install tensorflow

In [3]:
from PIL import Image

from keras import layers, models
from keras.applications.vgg16 import VGG16
from keras.applications.resnet import ResNet50
from keras.applications.resnet import ResNet
from keras.callbacks import EarlyStopping
from keras.models import Model
from keras.regularizers import l2

from skimage import io, color

from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import LeakyReLU

import glob
import json
import keras
import keras.backend as K
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import urllib.request

In [5]:
#######################################################
# CONTROL PANEL
#######################################################

target_var = "hdi" # either ("hdi" or "gdp")

In [6]:
# Loading X and Y

features = np.load("/content/drive/MyDrive/Thesis/imageset_500_500.npy") # adjust for colab

if target_var == 'hdi':
    labels = np.load("/content/drive/MyDrive/Thesis/target_HDI.npy") # adjust for colab
    
elif target_var == 'gdp':
    labels = np.load("/content/drive/MyDrive/Thesis/target_logGDP.npy") # adjust for colab

In [7]:
# Split 60% 20% 20%

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.4, random_state=1945, stratify=labels)

# Further split the training set into training and validation sets
X_test, X_val, y_test, y_val = train_test_split(
    X_test, y_test, test_size=0.5, random_state=1945, stratify=y_test)

In [8]:
# Float conversion to allow normalization
X_train=X_train.astype('float32')
X_test=X_test.astype('float32')
X_val=X_val.astype('float32')

# Normalization 
X_train=X_train/255.0
X_test=X_test/255.0
X_val=X_val/255.0

In [9]:
# Convert the numerical labels to one-hot encoded format
num_classes = 4
y_train_onehot = keras.utils.to_categorical((y_train-1), num_classes=num_classes)
y_val_onehot = keras.utils.to_categorical((y_val-1), num_classes=num_classes)
y_test_onehot = keras.utils.to_categorical((y_test-1), num_classes=num_classes)

VGG16 

In [ ]:
# Import VGG16
vgg_model = VGG16(include_top=False, input_shape=(500, 500, 3))

# Freezing VGG16 layers
for layer in vgg_model.layers:
    layer.trainable = False

## adding "custom" layers

## Flatten layer
flat_1 = layers.Flatten()(vgg_model.layers[-1].output)

## Dense layers
dense_1 = layers.Dense(256, activation='relu')(flat_1)
dense_2 = layers.Dense(128, activation='relu')(dense_1)
dense_3 = layers.Dense(64, activation='relu')(dense_2)
dense_4 = layers.Dense(32, activation='relu')(dense_3)
dense_f = layers.Dense(16, activation='relu')(dense_4)

#output layer with softmax 
output = layers.Dense(4, activation='softmax')(dense_f)

# define new model
tl_model = Model(inputs=vgg_model.inputs, outputs=output)

# summarize
tl_model.summary()

# compile model
tl_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Early stopping
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

# fit model
tl_trained = tl_model.fit(X_train, y_train_onehot, validation_data=(X_val, y_val_onehot), batch_size=64 ,epochs=100, verbose=1)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 500, 500, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 500, 500, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 500, 500, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 250, 250, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 250, 250, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 250, 250, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 125, 125, 128)     0     

KeyboardInterrupt: ignored

ResNet50

In [ ]:
# Import ResNet50
resnet_model = ResNet50(include_top=False, input_shape=(500, 500, 3))

# Freezing ResNet layers
for layer in resnet_model.layers:
    layer.trainable = False

## adding "custom" layers

## Flatten layer
flat_1 = layers.Flatten()(resnet_model.layers[-1].output)

## Dense layers
#dense_1 = layers.Dense(16384, activation='relu')(flat_1)
#dense_2 = layers.Dense(2048, activation='relu')(flat_1)
#dense_3 = layers.Dense(500, activation='relu')(flat_1)
dense_4 = layers.Dense(256, activation='relu')(flat_1)
dense_5 = layers.Dense(64, activation='relu')(dense_4)
dense_f = layers.Dense(16, activation='relu')(dense_5)

#output layer with softmax 
output = layers.Dense(4, activation='softmax')(dense_f)

# define new model
tl_resnet_model = Model(inputs=resnet_model.inputs, outputs=output)

# summarize
tl_resnet_model.summary()

# compile model
tl_resnet_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Early stopping
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

# fit model
tl_resnet_trained = tl_resnet_model.fit(X_train, y_train_onehot, validation_data=(X_val, y_val_onehot), batch_size=64 ,epochs=100, verbose=1)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 500, 500, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 506, 506, 3)  0           ['input_10[0][0]']               
                                                                                                  
 conv1_conv (Conv2D)            (None, 250, 250, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [ ]:
labels

array([3, 3, 3, ..., 4, 1, 1])

### Alexnet

In [50]:
X_train.permute(0, 3, 1, 2)

tensor([[[[0.2902, 0.2863, 0.3176,  ..., 0.1843, 0.2235, 0.2275],
          [0.2706, 0.2902, 0.3059,  ..., 0.2039, 0.2118, 0.2118],
          [0.2784, 0.2941, 0.2980,  ..., 0.2157, 0.1961, 0.2039],
          ...,
          [0.3216, 0.2588, 0.2824,  ..., 0.3569, 0.4784, 0.4980],
          [0.3098, 0.3137, 0.3608,  ..., 0.3333, 0.4588, 0.4941],
          [0.3294, 0.3490, 0.3804,  ..., 0.3608, 0.4510, 0.4745]],

         [[0.2863, 0.2824, 0.3137,  ..., 0.2314, 0.2706, 0.2745],
          [0.2667, 0.2863, 0.3020,  ..., 0.2510, 0.2588, 0.2588],
          [0.2745, 0.2902, 0.2941,  ..., 0.2588, 0.2471, 0.2510],
          ...,
          [0.3176, 0.2588, 0.2824,  ..., 0.3686, 0.4667, 0.4745],
          [0.3020, 0.3059, 0.3529,  ..., 0.3490, 0.4549, 0.4667],
          [0.3176, 0.3373, 0.3647,  ..., 0.3804, 0.4471, 0.4471]],

         [[0.2078, 0.2039, 0.2353,  ..., 0.1451, 0.1843, 0.1882],
          [0.1882, 0.2078, 0.2235,  ..., 0.1647, 0.1725, 0.1725],
          [0.1961, 0.2118, 0.2157,  ..., 0

In [10]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split, DataLoader
import torch.nn as nn


In [51]:
# Cuidado porque según la documentación de pytorch hace falta aplicar un prprocesado específico a las imágenes.
# Ver este enlace: https://pytorch.org/hub/pytorch_vision_alexnet/


# Data preprocessing according to chatGPT

# Pasando las features ya normalizados (/255) a tensor
X_train = torch.tensor(X_train)
X_val = torch.tensor(X_val)
X_test = torch.tensor(X_test)

# Pasando los targets (onehot) a tensor
y_train_onehot = torch.tensor(y_train_onehot)
y_val_onehot = torch.tensor(y_val_onehot)
y_test_onehot = torch.tensor(y_test_onehot)

# Zip de los objetos anteriores para que el dataloader se comporte de la manera deseada
# Al zip se le añade .permute para que la configuración de las dimensiones de los tensores 
# de entrada coincidan con las que espera el modelo.
Xy_train = list(zip(X_train.permute(0, 3, 1, 2), y_train_onehot))
Xy_val = list(zip(X_val.permute(0, 3, 1, 2), y_val_onehot))
Xy_test = list(zip(X_test.permute(0, 3, 1, 2), y_test_onehot))

# crea los dataloaders para cada conjunto
train_dataloader = DataLoader(Xy_train, batch_size=64, shuffle=True)
val_dataloader = DataLoader(Xy_val, batch_size=64, shuffle=False)
test_dataloader = DataLoader(Xy_test, batch_size=64, shuffle=False)


# Preparar los indices (train / val) -> From RNN task notebook

count = 0
for batch, (x, y) in enumerate(train_dataloader):
    count += (len(x*1))
    
train_indices_A = count


count = 0    
for batch, (x, y) in enumerate(val_dataloader):
    count += (len(x*1))
    
val_indices_A = count

count = 0    
for batch, (x, y) in enumerate(test_dataloader):
    count += (len(x*1))
    
val_test_A = count


<ipython-input-51-74724c07d5c2>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
<ipython-input-51-74724c07d5c2>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val = torch.tensor(X_val)
<ipython-input-51-74724c07d5c2>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
<ipython-input-51-74724c07d5c2>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.

In [52]:
AlexNet_model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=True)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [53]:
# Congelando los parámetros de la primera mitad del modelo (AlexNet_model.features)
# La otra mitad permanece descongelada (AlexNet_model.classifier)

for param in AlexNet_model.features.parameters():
    param.requires_grad = False

model = AlexNet_model

# verifica si una capa está congelada o descongelada
for name, param in model.named_parameters():
    if not param.requires_grad:
        print(f'Capa {name} está congelada')
    else:
        print(f'Capa {name} está descongelada')

# Include correct number of target classes (Originalmente hay mil incluidas)

AlexNet_model.classifier[6] = nn.Linear(1024, 4)

# Modify classifier 4 (En principio esto es opcional)

AlexNet_model.classifier[4] = nn.Linear(4096,1024)

Capa features.0.weight está congelada
Capa features.0.bias está congelada
Capa features.3.weight está congelada
Capa features.3.bias está congelada
Capa features.6.weight está congelada
Capa features.6.bias está congelada
Capa features.8.weight está congelada
Capa features.8.bias está congelada
Capa features.10.weight está congelada
Capa features.10.bias está congelada
Capa classifier.1.weight está descongelada
Capa classifier.1.bias está descongelada
Capa classifier.4.weight está descongelada
Capa classifier.4.bias está descongelada
Capa classifier.6.weight está descongelada
Capa classifier.6.bias está descongelada


In [54]:
# Resumen del modelo

AlexNet_model.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [102]:
def nn_converged_B(epoch: int, stop_after_epochs: int, validation_loss: torch.Tensor, model: torch.nn.Module) -> bool:
    converged = False
    # (Re)-start the epoch count with the first epoch or any improvement.
    if epoch == 0 or validation_loss < model.best_validation_loss:
        model.best_validation_loss = validation_loss
        model.epochs_since_last_improvement = 0
        model.best_model = copy.deepcopy(model.state_dict())
    else:
        model.epochs_since_last_improvement += 1

    # If no validation improvement over many epochs, stop training.
    if model.epochs_since_last_improvement > stop_after_epochs - 1:
        model.load_state_dict(model.best_model)
        converged = True
    return converged

In [65]:
import torch.optim as optim
import torch.nn as nn

device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cuda"
model = AlexNet_model.to(device)

#Define loss + optimizer
criterion = nn.CrossEntropyLoss() # El mismo que en los otros dos modelos
optimizer = optim.SGD(AlexNet_model.parameters(), lr=0.001, momentum=0.9)

In [140]:
def fit(
        train_loader: DataLoader,
        val_loader: DataLoader,
        train_indices: torch.Tensor,
        val_indices: torch.Tensor,
        num_epochs: int = 100,
        print_per_num_epochs: int = 5,
        convergence_num_epochs: int = 15,
    ) -> None:
# Run the training loop
    for epoch in range(num_epochs):
        print(f"epoch {epoch}")
        # Set the model in training mode for gradient evaluation
        model.train()
        # Set current loss value
        train_loss = 0.0
        correct_train = 0
        # Iterate over the DataLoader for training data
        for batch, (inputs_an, targets) in enumerate(train_loader):
            #if torch.cuda.is_available():
            inputs_an = inputs_an.to("cuda")
            targets = targets.to("cuda")
            # Zero the gradients
            optimizer.zero_grad()
            # Perform forward pass
            out = model(inputs_an)
            # Compute loss & accuracy

            #indices = torch.max(out, dim=1, keepdim=True)
            #discrete_out = torch.zeros_like(out)
            #discrete_out.scatter_(1, indices[1], 1)

            loss = criterion(out, targets).mean()
            #accuracy = (discrete_out == targets).float().mean()
            #print(targets)
            #print(out)
            #print(accuracy)
            # Perform backward pass
            loss.backward()
            # Perform optimization
            optimizer.step()
            # Add to the total of the training loss
            #train_accuracy += accuracy.item() * len(inputs_an)
            train_loss += loss.item() * len(inputs_an)
        # Once all training batches have been run, get the mean training loss
        #train_loss /= len(train_indices)
        train_loss /= train_indices
        # Set the model in evaluation mode so that gradients are not evaluated
        model.eval()
        val_loss = 0.0
        correct_val = 0
        with torch.no_grad():
            for i, (inputs_an, targets) in enumerate(val_loader):
                inputs_an = inputs_an.to("cuda")
                targets = targets.to("cuda")
                
                preds = model(inputs_an)

                # Loss
                loss = criterion(preds, targets).mean()
                val_loss += loss.item() * len(inputs_an)

                # Accuracy
                #accuracy = (predicted == targets)
                #val_accuracy += accuracy.item() * len(inputs_an)


            val_loss /= val_indices
            #val_accuracy /= val_indices

        if epoch % print_per_num_epochs == 0: 
            print(f"Train Loss after epoch: {epoch}: {train_loss}") 
            print(f"Validation loss after epoch: {epoch}: {val_loss}") 
            #print(f"Train accuracy after epoch: {epoch}: {train_accuracy}") 
            #print(f"Validation accuracy after epoch: {epoch}: {val_accuracy}") 

        # EARLY STOPPING - CEGADO
        #if nn_converged_B(epoch, convergence_num_epochs, val_loss, model): 
        #    print(f"Stopping after epoch {epoch} as validation loss was not improving further") 
        #   break 

In [ ]:
fit(train_loader = train_dataloader, val_loader = val_dataloader, train_indices = train_indices_A, val_indices = val_indices_A)

epoch 0
Train Loss after epoch: 0: 0.8536575057351492
Validation loss after epoch: 0: 0.8245534124197783
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
Train Loss after epoch: 5: 0.8222573174984474
Validation loss after epoch: 5: 0.7998132866028457
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
Train Loss after epoch: 10: 0.8050134473215274
Validation loss after epoch: 10: 0.7877727531434034
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
Train Loss after epoch: 15: 0.7646209873610396
Validation loss after epoch: 15: 0.7676318558277907
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
Train Loss after epoch: 20: 0.7608069023271886
Validation loss after epoch: 20: 0.7706054245170794
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
Train Loss after epoch: 25: 0.7100305646997157
Validation loss after epoch: 25: 0.7662129964512459
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
Train Loss after epoch: 30: 0.7016984004121486
Validation loss after epoch: 30: 0.7519390104051686
epoch 31
epoch 32
epoch 33
epoch 34
epoch 

In [49]:
for batch, (inputs, targets) in enumerate(train_dataloader):
  print(inputs.shape)
  print(targets.shape)
  break

torch.Size([64, 500, 500, 3])
torch.Size([64, 4])


In [63]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [118]:
tensor_float = torch.tensor([1.0, 2.0, 3.0, 2.0, 1.0])

# Obtenemos el índice del valor máximo
max_index = torch.argmax(tensor_float)

# Creamos un tensor de booleanos con True en la posición del valor máximo
tensor_bool = torch.eq(tensor_float, tensor_float[torch.argmax(tensor_float)]).float()


# Convertimos el tensor de booleanos en un tensor de ceros y unos
tensor_binary = tensor_bool.float()

print(tensor_binary)

tensor([False, False,  True, False, False])
tensor([0., 0., 1., 0., 0.])


In [121]:
a = torch.tensor([[ 1.3468e+00,  8.2564e-01, -3.1106e-01, -2.3416e+00],
        [ 4.7919e-01,  1.1269e+00, -6.0444e-02, -1.9886e+00],
        [ 2.4402e+00,  1.3219e+00, -8.7863e-01, -3.4789e+00],
        [-4.2046e+00, -2.6782e-01,  2.6880e+00,  1.5483e+00],
        [-1.6198e+00,  3.0126e+00,  3.8126e-01, -1.9735e+00],
        [-1.1049e+00,  1.3509e+00,  2.4179e-01, -7.5417e-01],
        [-9.2204e-01,  1.3891e+00,  1.8412e+00, -2.6736e+00],
        [-1.0776e+00,  2.0398e-02,  5.7748e-01, -5.0304e-01],
        [-1.8483e+00, -4.2850e-02,  7.5345e-01,  8.9233e-01],
        [ 1.5506e+00,  1.3335e+00, -7.8221e-01, -2.5273e+00],
        [-2.9566e+00, -2.0441e-01,  1.5702e+00,  1.3080e+00],
        [-4.3416e+00,  2.2675e+00,  9.0421e-01,  8.0281e-01],
        [-2.0504e+00,  1.0427e+00,  6.5517e-01,  6.5387e-02],
        [-3.6725e+00, -4.1544e-01,  1.6271e+00,  9.8365e-01],
        [-9.6558e-01,  1.0295e+00,  1.1233e+00, -9.2002e-01],
        [-9.6956e-01,  1.4521e+00,  5.2391e-01, -1.2358e+00],
        [-2.0454e+00, -1.2426e-01,  2.2888e+00, -3.3694e-01],
        [ 6.8609e-01,  1.4686e+00, -1.4211e-01, -2.6914e+00],
        [-4.7720e-01,  1.5395e+00,  8.2906e-01, -2.2274e+00],
        [-8.6643e-01,  8.9022e-01,  7.8947e-01, -1.1123e+00],
        [ 2.7497e-02,  3.1173e-01, -1.4744e-01, -7.5753e-01],
        [ 8.4915e-01,  2.5433e+00, -3.0871e-01, -3.1438e+00],
        [-3.4315e+00, -4.0053e-01,  2.7098e+00,  2.6060e-01],
        [ 1.2762e+00,  3.1163e-01, -6.2431e-03, -2.1250e+00],
        [-2.9375e+00, -1.0591e+00,  2.5584e+00,  7.0876e-01],
        [-4.0591e+00, -3.2206e-01,  3.3114e+00,  4.2015e-01],
        [-4.2747e+00, -1.0465e+00,  2.5839e+00,  2.4096e+00],
        [-4.8167e+00, -1.6270e+00,  2.9547e+00,  2.4055e+00],
        [-3.5385e+00,  3.8700e-01,  8.9815e-01,  1.6958e+00],
        [-2.5831e+00, -5.8904e-01,  1.8037e+00,  1.0412e+00],
        [-2.8786e+00, -2.0372e-01,  2.2054e+00,  2.1084e-01],
        [-4.1711e+00,  5.9310e-01,  3.3953e+00,  1.1264e-01],
        [-3.3624e+00,  1.9069e-01,  3.0444e+00,  7.1970e-02],
        [-2.2487e+00,  6.7301e-01,  1.1746e+00, -1.7859e-01],
        [-2.1653e+00,  3.6202e-01,  2.0640e+00, -8.0499e-01],
        [-2.3179e+00,  5.5355e-01,  8.5465e-01,  2.3528e-01],
        [-4.2128e+00,  6.0072e-01,  1.1938e+00,  8.4820e-01],
        [-1.3978e+00,  7.5284e-01,  1.8586e+00, -1.9992e+00],
        [-2.5835e+00,  5.3988e-01,  2.5313e+00, -1.2492e-01],
        [-4.6864e-01,  1.1190e+00, -3.8504e-02, -1.2285e+00],
        [-1.9579e+00,  2.5622e-01,  2.5567e+00, -1.0470e+00],
        [-3.0801e+00,  2.4219e-01,  2.2003e+00,  4.7775e-01],
        [-1.3178e+00,  9.0846e-01, -8.7199e-01,  4.9207e-02],
        [-1.9429e+00,  4.3202e-01,  5.2021e-01,  4.1096e-02],
        [ 9.7029e-02,  9.6023e-01, -3.6796e-01, -1.6643e+00],
        [-4.5852e+00, -1.5775e+00,  2.5484e+00,  3.0124e+00],
        [ 1.0438e+00,  2.0211e+00, -1.8641e-01, -3.3170e+00],
        [-2.9206e+00, -9.7813e-02,  1.8777e+00,  8.0069e-01],
        [-4.0910e+00, -1.4052e+00,  2.6880e+00,  1.9462e+00],
        [-8.7307e-01,  1.4536e+00,  5.7222e-01, -1.5045e+00],
        [ 1.7359e+00,  1.6690e+00, -6.9163e-01, -4.1355e+00],
        [-2.3620e+00, -3.9039e-04,  1.4656e+00,  4.5749e-01],
        [-1.1487e+00,  2.8058e-01,  2.0239e-01,  2.2004e-01],
        [ 1.0791e+00,  9.9053e-01, -2.2999e-01, -2.6491e+00],
        [-8.8892e-01,  1.1808e+00,  3.5497e-01, -1.0615e+00],
        [-3.8006e+00,  9.8551e-01,  1.1817e+00,  1.4784e+00],
        [-1.7489e+00,  5.3113e-01,  6.9220e-01, -2.9093e-01],
        [ 4.7737e-01,  1.0107e+00,  1.0295e+00, -3.2203e+00],
        [-3.0578e+00,  1.0582e+00, -4.2955e-01,  1.5401e+00],
        [ 1.3502e+00,  1.6221e+00, -6.3533e-01, -2.8572e+00],
        [-4.6335e+00, -1.4216e+00,  1.9418e+00,  3.0482e+00],
        [-2.9345e-01,  1.4985e-01,  1.0530e+00, -1.6788e+00],
        [-9.3173e-01,  1.0552e+00,  4.0262e-01, -1.0204e+00],
        [-2.3596e+00,  2.2867e-01,  2.5723e+00, -3.8379e-01]])

In [126]:
indices = torch.max(a, dim=1, keepdim=True)
tensor_one_hot = torch.zeros_like(a)
tensor_one_hot.scatter_(1, indices[1], 1)
tensor_one_hot

tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0.,